In [1]:
# Import necessary libraries
import pandas as pd
import numpy as np
import requests
import json
import warnings
warnings.filterwarnings("ignore")

from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
# Step 1: Make the initial API call to get the total number of pages
api_key = '8265bd1679663a7ea12ac168da84d2e8'
initial_url = 'https://api.themoviedb.org/3/movie/top_rated'
params = {'api_key': api_key, 'language': 'en-US', 'page': 1}
moview_descrtiption_api = requests.get(initial_url, params=params)
total_pages = json.loads(moview_descrtiption_api.text)['total_pages']

### The above cell initializes the process by making the initial API call to get the total number of pages.

In [3]:
# Step 2: Initialize an empty DataFrame
df = pd.DataFrame()

# Step 3: Loop through each page and fetch movie data
for page_number in range(1, total_pages + 1):
    # Make API call for each page
    page_url = 'https://api.themoviedb.org/3/movie/top_rated'
    page_params = {'api_key': api_key, 'language': 'en-US', 'page': page_number}
    page_request = requests.get(page_url, params=page_params)

    # Load JSON data from the API response
    page_data = json.loads(page_request.text)

    # Get the number of movies on the current page
    movies_per_page = len(page_data['results'])

    # Print information about the current page
    print(f"Reading {movies_per_page} movies data from page {page_number}")

    # Loop through each movie on the current page and append data to DataFrame
    for i in range(movies_per_page):
        row = {
            'movie_title': page_data['results'][i]['original_title'],
            'genre': page_data['results'][i]['genre_ids'],
            'overview': page_data['results'][i]['overview']
        }
        df = df.append(row, ignore_index=True)

Reading 20 movies data from page 1
Reading 20 movies data from page 2
Reading 20 movies data from page 3
Reading 20 movies data from page 4
Reading 20 movies data from page 5
Reading 20 movies data from page 6
Reading 20 movies data from page 7
Reading 20 movies data from page 8
Reading 20 movies data from page 9
Reading 20 movies data from page 10
Reading 20 movies data from page 11
Reading 20 movies data from page 12
Reading 20 movies data from page 13
Reading 20 movies data from page 14
Reading 20 movies data from page 15
Reading 20 movies data from page 16
Reading 20 movies data from page 17
Reading 20 movies data from page 18
Reading 20 movies data from page 19
Reading 20 movies data from page 20
Reading 20 movies data from page 21
Reading 20 movies data from page 22
Reading 20 movies data from page 23
Reading 20 movies data from page 24
Reading 20 movies data from page 25
Reading 20 movies data from page 26
Reading 20 movies data from page 27
Reading 20 movies data from page 28
R

In [4]:
genre_url='https://api.themoviedb.org/3/genre/movie/list'
genres = requests.get(genre_url, params=params)
all_gen = pd.json_normalize(json.loads(genres.text)['genres'])


def get_genre(genre_id_list):
  genre_name_list=[]
  for id in genre_id_list:
    genre = all_gen[all_gen['id']==12]['name'].iloc[0]
    genre_name_list.append(genre)
  return genre_name_list


for i in df.index:
  genre_id_list = df.at[i,'genre']
  df.at[i,'genre']=get_genre(genre_id_list)

In [5]:
df.head()

,movie_title,genre,overview
0,The Shawshank Redemption,"[Adventure, Adventure]",Framed in the 1940s for the double murder of h...
1,The Godfather,"[Adventure, Adventure]","Spanning the years 1945 to 1955, a chronicle o..."
2,The Godfather Part II,"[Adventure, Adventure]",In the continuing saga of the Corleone crime f...
3,Schindler's List,"[Adventure, Adventure, Adventure]",The true story of how businessman Oskar Schind...
4,12 Angry Men,[Adventure],The defense and the prosecution have rested an...


In [6]:
df_copy=df.copy()
df.head()

,movie_title,genre,overview
0,The Shawshank Redemption,"[Adventure, Adventure]",Framed in the 1940s for the double murder of h...
1,The Godfather,"[Adventure, Adventure]","Spanning the years 1945 to 1955, a chronicle o..."
2,The Godfather Part II,"[Adventure, Adventure]",In the continuing saga of the Corleone crime f...
3,Schindler's List,"[Adventure, Adventure, Adventure]",The true story of how businessman Oskar Schind...
4,12 Angry Men,[Adventure],The defense and the prosecution have rested an...


In [7]:
import os
os.chdir('/content/drive/MyDrive/01. DSML_ML Algorithms/Data sets')

In [8]:
df_copy.to_csv('/content/drive/MyDrive/01. DSML_ML Algorithms/Data sets/NLP_TMDB_movie_description.csv')
# drive.umount('/content/drive')

In [9]:
df.sample(1).values

array([['The Time Machine',
        list(['Adventure', 'Adventure', 'Adventure']),
        'Hoping to alter the events of the past, a 19th century inventor instead travels 800,000 years into the future, where he finds mankind divided into two warring races.']],
      dtype=object)

**Remove \<HMTL> tags:**  
- Untilize an  regex `<.*?>` to remove the tags by retaining the text in between the tags.

In [10]:
import re

def remove_html_tags(text):
  pattern = re.compile('<.*?>')
  return pattern.sub(r'',text)

In [11]:
df['overview']=df['overview'].apply(remove_html_tags)

**Remove punctuations:**  
- Utilize the  `string.punctuation` module to compile a set of unwanted punctuation characters.
- Apply the `translate` method in conjunction with `maketrans` to eliminate the specified punctuations from the text.

In [13]:
import string
exclude = string.punctuation

In [16]:
df['overview']=df['overview'].apply(lambda x: x.translate(str.maketrans('','',exclude)))

**Handling spelling mistakes**
- Utilize the `TextBlob` module to convret miss-spelled words into correct words.

In [30]:
#TextBlob uses WordNetLemmatizer may not handle all types of spelling errors.
'''from textblob import TextBlob
df['overview']=df['overview'].apply(lambda text: TextBlob(text).correct().string)'''

"from textblob import TextBlob\ndf['overview']=df['overview'].apply(lambda text: TextBlob(text).correct().string)"

**Handling STOP-WORDS**  
- `nltk` provides us with a library called `stopwords` which will help you to remove the stop wrods in a specific language
- `nltk.corpus.stopwords` contains around **16 languages** related stopwords

In [36]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [38]:
from nltk.corpus import stopwords
english_stopwords=stopwords.words('english')

In [51]:
#retuns the string after removing the stopwords
#Filter: filter(function,iterable)
#Ex: list(filter(lambda x: x % 2 != 0, numbers)) - filters out the number that satisfy the condition/function (odd number in this case)
def remove_stopwords(text):
  return ' '.join(list(filter(lambda word: word not in english_stopwords,text.split())))

In [52]:
df['overview']=df['overview'].apply(remove_stopwords)

**Tokenization**  
